# 1. Import Libraries

In [4]:
!pip install folium

     |████████████████████████████████| 102kB 10.2MB/s ta 0:00:01


In [10]:
!pip install geocoder

     |████████████████████████████████| 102kB 8.6MB/s ta 0:00:011


In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


# 2. Scrap data from Wikipedia page into a DataFrame

In [19]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:South_Jakarta").text

In [20]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [21]:
# create a list to store neighborhood data
neighborhoodList = []

In [22]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [23]:
# create a new DataFrame from the list
jkt_df = pd.DataFrame({"Neighborhood": neighborhoodList})

jkt_df.head()

,Neighborhood
0,South Jakarta
1,AIS Indonesia
2,Atma Jaya Catholic University of Indonesia
3,Balai Sarbini
4,Bintaro Jaya


In [24]:
# print the number of rows of the dataframe
jkt_df.shape

(63, 1)

# 3. Get the geographical coordinates

In [26]:

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, South Jakarta, Indonesia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [29]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in jkt_df["Neighborhood"].tolist() ]

In [30]:
coords

[[-6.253739999999937, 106.79731000000004],
 [-6.278239999999926, 106.82858000000004],
 [-6.194169999999929, 106.84914000000003],
 [-6.253739999999937, 106.79731000000004],
 [-6.236189999999965, 106.75495000000006],
 [-6.106906770616363, 106.79343662091716],
 [-6.253739999999937, 106.79731000000004],
 [-6.231999999999971, 106.82066000000003],
 [-6.252749999999935, 106.86463000000003],
 [-6.2905099999999265, 106.79491000000007],
 [-6.253739999999937, 106.79731000000004],
 [-6.243109999999945, 106.84359000000006],
 [-6.255259999999964, 106.85533000000004],
 [-6.2925299999999424, 106.79313000000008],
 [-6.3397993720856345, 106.85197822672478],
 [-6.245719999999949, 106.78393000000005],
 [-6.13659999999993, 106.82145000000008],
 [-6.343079999999929, 106.81745000000006],
 [-6.286059999999964, 106.78253000000007],
 [-6.219255358702095, 106.83151325951332],
 [-6.223079163678003, 106.82320218931301],
 [-6.279489999999953, 106.81522000000007],
 [-6.227639999999951, 106.82692000000009],
 [-6.2514

In [31]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [54]:
# merge the coordinates into the original dataframe
jkt_df['Latitude'] = df_coords['Latitude']
jkt_df['Longitude'] = df_coords['Longitude']

In [55]:
# check the neighborhoods and the coordinates
print(jkt_df.shape)
jkt_df

(63, 3)


,Neighborhood,Latitude,Longitude
0,South Jakarta,-6.253740,106.797310
1,AIS Indonesia,-6.278240,106.828580
2,Atma Jaya Catholic University of Indonesia,-6.194170,106.849140
3,Balai Sarbini,-6.253740,106.797310
4,Bintaro Jaya,-6.236190,106.754950
5,Blok M,-6.106907,106.793437
6,BTPN Towers,-6.253740,106.797310
7,Capital Place Jakarta,-6.232000,106.820660
8,Cawang railway station,-6.252750,106.864630
9,Cilandak,-6.290510,106.794910


In [56]:
# save the DataFrame as CSV file
jkt_df.to_csv("kl_df.csv", index=False)

# 4. Create a map of South Jakarta with neighborhoods superimposed on top

In [57]:
# get the coordinates of South Jakarta
address = 'South Jakarta, Indonesia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of South Jakarta, Indonesia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of South Jakarta, Indonesia -6.28381815, 106.80486349194814.


In [36]:
# create map of South Jakarta using latitude and longitude values
map_jkt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(jkt_df['Latitude'], jkt_df['Longitude'], jkt_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_jkt)  
    
map_jkt

In [37]:
# save the map as HTML file
map_jkt.save('map_jkt.html')

# 5. Use the Foursquare API to explore the neighborhoods

In [69]:
CLIENT_ID = '0ZTCNYI5ZNXDLOCPGXCW4XVLJVM0NEHK0XILEUOVKGFII0VV' # your Foursquare ID
CLIENT_SECRET = 'LUGWRZD0AQXRRP4HJHNAVCTZCYMBBK34XQX3LO3V5YOZF4HN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0ZTCNYI5ZNXDLOCPGXCW4XVLJVM0NEHK0XILEUOVKGFII0VV
CLIENT_SECRET:LUGWRZD0AQXRRP4HJHNAVCTZCYMBBK34XQX3LO3V5YOZF4HN


In [71]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(jkt_df['Latitude'], jkt_df['Longitude'], jkt_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [72]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5854, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,South Jakarta,-6.25374,106.79731,Nasi Uduk Pak Joko Putro,-6.254641,106.796036,Indonesian Restaurant
1,South Jakarta,-6.25374,106.79731,Sopo Ngiro Japanese Food,-6.251255,106.797441,Japanese Restaurant
2,South Jakarta,-6.25374,106.79731,Chung Gi Wa 청기와,-6.252591,106.800207,Korean Restaurant
3,South Jakarta,-6.25374,106.79731,Paviliun 28,-6.251501,106.794669,General Entertainment
4,South Jakarta,-6.25374,106.79731,Nasi Campur Kenanga,-6.253358,106.800189,BBQ Joint


Let's check how many venues were returned for each neighorhood

In [73]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
AIS Indonesia,100,100,100,100,100,100
Atma Jaya Catholic University of Indonesia,100,100,100,100,100,100
BTPN Towers,100,100,100,100,100,100
Balai Sarbini,100,100,100,100,100,100
Bintaro Jaya,56,56,56,56,56,56
Blok M,99,99,99,99,99,99
Capital Place Jakarta,100,100,100,100,100,100
Cawang railway station,100,100,100,100,100,100
Cilandak,100,100,100,100,100,100


Let's find out how many unique categories can be curated from all the returned venues

In [74]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 241 uniques categories.


In [75]:
venues_df['VenueCategory'].unique()[:50]

array(['Indonesian Restaurant', 'Japanese Restaurant',
       'Korean Restaurant', 'General Entertainment', 'BBQ Joint',
       'Fruit & Vegetable Store', "Women's Store", 'Seafood Restaurant',
       'Coffee Shop', 'Cosmetics Shop', 'Hotel', 'Board Shop',
       'Paper / Office Supplies Store', 'Burger Joint', 'Spa',
       'Snack Place', 'Italian Restaurant', 'Gourmet Shop',
       'Comfort Food Restaurant', 'Noodle House', 'Food Truck',
       'Restaurant', 'Sushi Restaurant', 'Padangnese Restaurant',
       'Fried Chicken Joint', 'Beer Garden', 'Supermarket',
       'Grocery Store', 'Furniture / Home Store', 'Lounge',
       'Dance Studio', 'Steakhouse', 'Food & Drink Shop', 'Shopping Mall',
       'Soup Place', 'Art Gallery', 'German Restaurant', 'Bistro',
       'Automotive Shop', 'Bakery', 'Playground', 'Asian Restaurant',
       'Electronics Store', 'Plaza', 'Javanese Restaurant',
       'Clothing Store', 'Salon / Barbershop', 'Bar', 'Movie Theater',
       'Bookstore'], dtype=

In [76]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

# 6. Analyze Each Neighborhood

In [77]:
jkt_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jkt_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jkt_onehot.columns[-1]] + list(jkt_onehot.columns[:-1])
jkt_onehot = jkt_onehot[fixed_columns]

print(jkt_onehot.shape)
jkt_onehot.head()

(5854, 242)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Butcher,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Library,Light Rail Station,Lighthouse,Lingerie Store,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,South Jakarta,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [78]:
jkt_grouped = jkt_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(jkt_grouped.shape)
jkt_grouped

(63, 242)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Butcher,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Factory,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Library,Light Rail Station,Lighthouse,Lingerie Store,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Great Outdoors,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pub,Public Art,Racetrack,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo
0,AIS Indonesia,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.00,0.030000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.010000,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.00,0.010000,0.01,0.010000,0.010000,0.00,0.010000,0.000000,0.00,0.000000,0.030000,0.01,0.000000,0.00,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.01,0.130000,0.00,0

In [80]:
len(jkt_grouped[jkt_grouped["Shopping Mall"] > 0])

43

Create a new DataFrame for Shopping Mall data only

In [81]:
jkt_mall = jkt_grouped[["Neighborhoods","Shopping Mall"]]

In [82]:
jkt_mall.head()

,Neighborhoods,Shopping Mall
0,AIS Indonesia,0.00
1,Atma Jaya Catholic University of Indonesia,0.00
2,BTPN Towers,0.01
3,Balai Sarbini,0.01
4,Bintaro Jaya,0.00


# 7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in South Jakarta into 3 clusters

In [83]:
kclusters = 3

jkt_clustering = jkt_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jkt_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 1, 2, 2, 1, 1, 1, 1], dtype=int32)

In [85]:
jkt_merged = jkt_mall.copy()

# add clustering labels
jkt_merged["Cluster Labels"] = kmeans.labels_

In [86]:
jkt_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
jkt_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,AIS Indonesia,0.00,2
1,Atma Jaya Catholic University of Indonesia,0.00,2
2,BTPN Towers,0.01,1
3,Balai Sarbini,0.01,1
4,Bintaro Jaya,0.00,2


In [87]:
jkt_merged = jkt_merged.join(jkt_df.set_index("Neighborhood"), on="Neighborhood")

print(jkt_merged.shape)
jkt_merged.head()

(63, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,AIS Indonesia,0.00,2,-6.27824,106.82858
1,Atma Jaya Catholic University of Indonesia,0.00,2,-6.19417,106.84914
2,BTPN Towers,0.01,1,-6.25374,106.79731
3,Balai Sarbini,0.01,1,-6.25374,106.79731
4,Bintaro Jaya,0.00,2,-6.23619,106.75495


In [88]:
print(jkt_merged.shape)
jkt_merged.sort_values(["Cluster Labels"], inplace=True)
jkt_merged

(63, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
23,Kebayoran Lama,0.040000,0,-6.251560,106.781510
44,Pondok Indah Mall,0.030000,0,-6.266260,106.783350
43,Pondok Indah,0.030000,0,-6.268570,106.780560
41,Perbanas Institute,0.040000,0,-6.216910,106.829520
48,Setiabudi,0.040000,0,-6.221240,106.825280
35,Pacific Place Jakarta,0.030000,0,-6.224490,106.810490
33,Mega Kuningan,0.050000,0,-6.224470,106.825570
61,The Plaza Semanggi,0.030000,0,-6.220990,106.813750
27,Kuningan City,0.040000,0,-6.225130,106.829740
20,Jalan Prof. Dr. Satrio,0.040000,0,-6.223079,106.823202


Finally, let's visualize the resulting clusters

In [90]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jkt_merged['Latitude'], jkt_merged['Longitude'], jkt_merged['Neighborhood'], jkt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [91]:
map_clusters.save('map_clusters.html')

# 8. Examine Clusters

Cluster 0

In [92]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
23,Kebayoran Lama,0.040000,0,-6.251560,106.781510
44,Pondok Indah Mall,0.030000,0,-6.266260,106.783350
43,Pondok Indah,0.030000,0,-6.268570,106.780560
41,Perbanas Institute,0.040000,0,-6.216910,106.829520
48,Setiabudi,0.040000,0,-6.221240,106.825280
35,Pacific Place Jakarta,0.030000,0,-6.224490,106.810490
33,Mega Kuningan,0.050000,0,-6.224470,106.825570
61,The Plaza Semanggi,0.030000,0,-6.220990,106.813750
27,Kuningan City,0.040000,0,-6.225130,106.829740
20,Jalan Prof. Dr. Satrio,0.040000,0,-6.223079,106.823202


Cluster 1

In [93]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
14,Gandaria City,0.010000,1,-6.24572,106.78393
60,The Peak Twin Towers,0.010000,1,-6.25374,106.79731
2,BTPN Towers,0.010000,1,-6.25374,106.79731
3,Balai Sarbini,0.010000,1,-6.25374,106.79731
39,Patung Pemuda Membangun,0.010000,1,-6.18268,106.75190
49,Setiabudi Astra MRT station,0.020000,1,-6.20926,106.82143
36,"Pancoran, South Jakarta",0.020000,1,-6.25548,106.84391
58,"Tebet, South Jakarta",0.010000,1,-6.22939,106.84971
34,New Zealand Independent School Jakarta,0.010000,1,-6.26471,106.81693
6,Capital Place Jakarta,0.020000,1,-6.23200,106.82066


Cluster 2

In [94]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
50,Setu Babakan Betawi Cultural Village,0.0,2,-6.356120,106.693270
51,"Singapore International School, Indonesia",0.0,2,-6.197270,106.758760
47,Senayan MRT station,0.0,2,-6.166240,106.876570
53,State University of Jakarta,0.0,2,-6.169880,106.872460
0,AIS Indonesia,0.0,2,-6.278240,106.828580
40,Pejaten Village,0.0,2,-6.280940,106.828290
38,Pasar Minggu railway station,0.0,2,-6.292650,106.827910
37,Pasar Minggu Baru railway station,0.0,2,-6.171480,106.826490
29,Lenteng Agung railway station,0.0,2,-6.323830,106.839080
28,Lebak Bulus Grab MRT station,0.0,2,-6.289210,106.776030
